In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow
import keras
import numpy as np

Using TensorFlow backend.


In [3]:
df=pd.read_csv("F:/IIT/Github/TensorFlow-Pokemon-Course/pokemon.csv")

df.columns

df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

df.columns

Index(['isLegendary', 'Generation', 'Type_1', 'Type_2', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Color', 'Egg_Group_1',
       'Height_m', 'Weight_kg', 'Body_Style'],
      dtype='object')

In [4]:
df['isLegendary'] = df['isLegendary'].astype(int)
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)
        df = df.drop(i, axis=1)
    return(df)
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])


In [6]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)


In [8]:
df_train, df_test = train_test_splitter(df, 'Generation')


In [9]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

In [10]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')


In [11]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

In [12]:

train_data, test_data = data_normalizer(train_data, test_data)

length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))


In [13]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [14]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
570/570 [==============================] - 0s 297us/step - loss: 0.5527 - accuracy: 0.9175
Epoch 2/400
570/570 [==============================] - 0s 124us/step - loss: 0.4262 - accuracy: 0.9263
Epoch 3/400
570/570 [==============================] - 0s 123us/step - loss: 0.3620 - accuracy: 0.9263
Epoch 4/400
570/570 [==============================] - 0s 128us/step - loss: 0.3258 - accuracy: 0.9263
Epoch 5/400
570/570 [==============================] - 0s 123us/step - loss: 0.3035 - accuracy: 0.9263
Epoch 6/400
570/570 [==============================] - 0s 98us/step - loss: 0.2886 - accuracy: 0.9263
Epoch 7/400
570/570 [==============================] - 0s 68us/step - loss: 0.2778 - accuracy: 0.9263
Epoch 8/400
570/570 [==============================] - 0s 70us/step - loss: 0.2695 - accuracy: 0.9263
Epoch 9/400
570/570 [==============================] - 0s 73us/step - loss: 0.2626 - accuracy: 0.9263
Epoch 10/400
570/570 [==============================] - 0s 73us/step - loss: 

In [15]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')


151/151 [==============================] - 0s 198us/step
Our test accuracy was 0.9801324605941772


In [16]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)
    
predictor(test_data, test_labels, 149)


This was correctly predicted to be a "1"!


In [17]:
predictor(test_data, test_labels, 12)

This was correctly predicted to be a "0"!
